# Code Review
이 코드는 https://wikidocs.net/21667 사이트의 코드를 가져와 리뷰한 것입니다.

In [1]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [76]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [77]:
lines= pd.read_csv('fra.txt', names=['src', 'tar'], sep='\t')
len(lines)

178009

In [78]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:1000] # 6만개만 저장
lines.sample(10)

,src,tar
I'm sorry.,Excusez-moi.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
I'm full.,Je suis rassasié !,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
It's work.,C'est du boulot.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
I'll live.,Je vivrai.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Who's he?,Qui est-il ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Let me go.,Laisse-moi m'en aller !,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
Get lost!,Va au diable !,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
I've lost.,J'ai perdu.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
I saw you.,Je te vis.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Am I late?,Suis-je en retard ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [79]:
lines.tar=lines.src

In [80]:
lines.src=lines.index
lines.sample(10)

,src,tar
Go away.,Go away.,Pars !
No way!,No way!,En aucune manière !
Can we go?,Can we go?,Pouvons-nous partir ?
Of course!,Of course!,Pour sûr.
May I go?,May I go?,Puis-je partir ?
Move over.,Move over.,Poussez-vous.
I've won.,I've won.,J'ai gagné.
I was new.,I was new.,J'étais nouveau.
Welcome.,Welcome.,Bienvenue !
Get out.,Get out.,Sors.


In [81]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
I hurried.,I hurried.,\t Je me suis dépêchée. \n
I'm cold.,I'm cold.,\t J'ai froid. \n
Take mine.,Take mine.,\t Prends la mienne. \n
I retired.,I retired.,\t J'ai pris ma retraite. \n
I try.,I try.,\t J'essaye. \n
Call Tom.,Call Tom.,\t Appelez Tom. \n
Hi.,Hi.,\t Salut. \n
He's mine.,He's mine.,\t Il est à moi. \n
She's hot.,She's hot.,\t Elle est très attirante. \n
Get a job.,Get a job.,\t Trouve un emploi ! \n


In [82]:
lines.head()

,src,tar
Go.,Go.,\t Va ! \n
Hi.,Hi.,\t Salut ! \n
Hi.,Hi.,\t Salut. \n
Run!,Run!,\t Cours ! \n
Run!,Run!,\t Courez ! \n


In [83]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [84]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

64
78


In [85]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '\xa0', 'À', 'Ç', 'É', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ô', 'û', '\u2009']


In [91]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '$': 3, "'": 4, ',': 5, '.': 6, '0': 7, '1': 8, '3': 9, '5': 10, '8': 11, '9': 12, ':': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'R': 31, 'S': 32, 'T': 33, 'U': 34, 'V': 35, 'W': 36, 'Y': 37, 'a': 38, 'b': 39, 'c': 40, 'd': 41, 'e': 42, 'f': 43, 'g': 44, 'h': 45, 'i': 46, 'j': 47, 'k': 48, 'l': 49, 'm': 50, 'n': 51, 'o': 52, 'p': 53, 'q': 54, 'r': 55, 's': 56, 't': 57, 'u': 58, 'v': 59, 'w': 60, 'x': 61, 'y': 62, 'z': 63}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '3': 11, '8': 12, '9': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'Q': 31, 'R': 32, 'S': 33, 'T': 34, 'U': 35, 'V': 36, 'a': 37, 'b': 38, 'c': 39, 'd': 40, 'e': 41, 'f': 42, 'g': 43, 'h': 44, 'i': 45, 'j': 46, 'k': 47, 'l': 48, 'm': 49, 'n': 5

In [87]:
tar_to_index['\t']=tar_to_index[' ']

In [88]:
del tar_to_index[' ']

In [92]:
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[21, 52, 6], [22, 46, 6], [22, 46, 6], [31, 58, 51, 2], [31, 58, 51, 2]]


In [93]:
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 36, 37, 3, 4, 3, 2], [1, 3, 33, 37, 48, 57, 56, 3, 4, 3, 2], [1, 3, 33, 37, 48, 57, 56, 8, 3, 2], [1, 3, 17, 51, 57, 54, 55, 77, 4, 3, 2], [1, 3, 17, 51, 57, 54, 41, 61, 77, 4, 3, 2]]


In [94]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 36, 37, 3, 4, 3, 2], [3, 33, 37, 48, 57, 56, 3, 4, 3, 2], [3, 33, 37, 48, 57, 56, 8, 3, 2], [3, 17, 51, 57, 54, 55, 77, 4, 3, 2], [3, 17, 51, 57, 54, 41, 61, 77, 4, 3, 2]]


In [95]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

10
35


In [96]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [97]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [99]:
print(len(encoder_input[0]))
print(encoder_input[0])

10
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [

In [61]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [62]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [63]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [64]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 28s 36ms/sample - loss: 2.4131 - val_loss: 1.9179
Epoch 2/50
800/800 [==============================] - 7s 9ms/sample - loss: 1.6487 - val_loss: 1.9536
Epoch 3/50
800/800 [==============================] - 5s 6ms/sample - loss: 1.5201 - val_loss: 1.7782
Epoch 4/50
800/800 [==============================] - 5s 6ms/sample - loss: 1.4645 - val_loss: 1.8822
Epoch 5/50
800/800 [==============================] - 4s 5ms/sample - loss: 1.4514 - val_loss: 1.5768
Epoch 6/50
800/800 [==============================] - 5s 7ms/sample - loss: 1.3886 - val_loss: 1.5211
Epoch 7/50
800/800 [==============================] - 5s 6ms/sample - loss: 1.3514 - val_loss: 1.4909
Epoch 8/50
800/800 [==============================] - 5s 7ms/sample - loss: 1.3345 - val_loss: 1.4547
Epoch 9/50
800/800 [==============================] - 4s 5ms/sample - loss: 1.2827 - val_loss: 1.4384
Epoch 10/50
800/800 [=============

In [65]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [66]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [67]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [68]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence